In [ ]:
# importing modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import util
import ast
import pickle
from sklearn.utils import shuffle
import seaborn as sn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import math
from scipy.spatial.distance import cosine

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
import pyLDAvis.gensim
from numpy import linalg as LA

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier as ada


In [ ]:
data = pd.read_csv("./Final_Data/all_pairs_science.csv")
df = pd.read_csv("./Final_Data/all_pairs_science_sc_nv.csv")
    
data["SC_score"] = df ["SC_score"]
data["Novelty_score"] = df ["Novelty_score"]



In [ ]:
data.head(5)

In [ ]:
data.lda_topicss1 =data.lda_topicss1.apply(lambda x: ast.literal_eval(x))
data.lda_topicss2 =data.lda_topicss2.apply(lambda x: ast.literal_eval(x))
# type(data.lda_topicss1[0])

In [ ]:
#  generating the target variable
# data["vaild_resource"] = data["collection_id1"] == data['collection_id2']        

In [ ]:
# loading doc2vec and word2vec models and function to return doc embedding
model1= Doc2Vec.load("./Final_Data/doc2vec_100dim_science.model")
# model2 =Doc2Vec.load("doc2vec_50dim.model")

def document_embeddings(doc,Embedding_size=100):
    test_data = word_tokenize(doc.lower())
    return model1.infer_vector(test_data)            
#     if(Embedding_size==100):
#         return model1.infer_vector(test_data)        
#     elif (Embedding_size == 50):
#         return model2.infer_vector(test_data)

In [ ]:
# generating doc embedding for pairwise resources
data["embeddings_text1"] = data.Summarizations1.apply(lambda x: document_embeddings(x))
data["embeddings_text2"] = data.Summarizations2.apply(lambda x: document_embeddings(x))

In [ ]:
# loading word2vec and function to retuen word embeddings

word2vec = Word2Vec.load('./Final_Data/Word2Vec_100dim_science')

def Word2doc(x):
    words=x.split()
    emb=np.zeros(100)
    for word in words:
        if(word in word2vec.wv.vocab):
            emb = np.add(emb,word2vec[word])
    return emb/len(words)   

In [ ]:
# generating word embedding for pairwise resources

data["word_emb1"]=data.Summarizations1.apply(lambda x :Word2doc(x))
data["word_emb2"]=data.Summarizations2.apply(lambda x :Word2doc(x))

In [ ]:
# initializing cosine similarity between pairs of document embedding and word embeddings
data["doc_cosine"] = 0
data["word_cosine"] =0

In [ ]:
# functions to calcualte cosine similarity between pairs of document embedding and word embeddings

def cosine_angle_word(x):
    if(LA.norm(x["word_emb1"])!=0 and LA.norm(x["word_emb2"])!=0):
        return (np.dot(np.array(x["word_emb1"]),np.array(x["word_emb2"]))/(LA.norm(x["word_emb1"]) * LA.norm(x["word_emb2"])))
    else:
        return 1
    
def cosine_angle_doc(x):
    if(LA.norm(x["embeddings_text2"])!=0 and LA.norm(x["embeddings_text1"])!=0):
        return (np.dot(np.array(x["embeddings_text1"]),np.array(x["embeddings_text2"]))/(LA.norm(x["embeddings_text1"]) * LA.norm(x["embeddings_text2"])))
    else:
        return 1  


In [ ]:
# calcualting cosine similarity between pairs of document embedding and word embeddings

data["doc_cosine"]= data.apply(cosine_angle_doc,axis=1)
data["word_cosine"]= data.apply(cosine_angle_word,axis=1)


In [ ]:
# encoding the target table 

# def encoding(boolean):
#     if(boolean):
#         return 1
#     return 0

# data.vaild_resource = data.vaild_resource.apply(lambda x:encoding(x))

In [ ]:
# # splitting document embeddings to send as input for svm

# columns=["docEmb1_" + str(i) for i in range(1,101)]
# data[columns] = pd.DataFrame(data.embeddings_text1.values.tolist(), index= data.index)


# columns=["docEmb2_" + str(i) for i in range(1,101)]
# data[columns] = pd.DataFrame(data.embeddings_text2.values.tolist(), index= data.index)


In [ ]:
# # splitting word embeddings to send as input for svm

# columns=["wordEmb1_" + str(i) for i in range(1,101)]
# data[columns] = pd.DataFrame(data.word_emb1.values.tolist(), index= data.index)

# columns=["wordEmb2_" + str(i) for i in range(1,101)]
# data[columns] = pd.DataFrame(data.word_emb2.values.tolist(), index= data.index)


In [ ]:
# lda = gensim.models.ldamodel.LdaModel
# fileObject = open('./PickelFiles/lda_dictionary.model','rb')  
# dictionary = pickle.load(fileObject)
# ldamodel = lda.load('./PickelFiles/lda_25_sc.model')

In [ ]:
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation) 
# lemma = WordNetLemmatizer()
# def clean(doc):
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized

In [ ]:
# data["clean1"]=data.text1.apply(lambda x: clean(x))
# data["clean2"]=data.text2.apply(lambda x: clean(x))
# data.clean1= data.clean1.apply(lambda x: x.split())
# data.clean2 = data.clean2.apply(lambda x: x.split())

In [ ]:
# data['clean_matrix1'] = data.clean1.apply(lambda x: dictionary.doc2bow(x))
# data['clean_matrix2'] = data.clean2.apply(lambda x: dictionary.doc2bow(x))


In [ ]:
# data['lda_topic1'] = data.clean_matrix1.apply(lambda x: ldamodel.get_document_topics(x,per_word_topics=True)[0])
# data['lda_topic2'] = data.clean_matrix2.apply(lambda x: ldamodel.get_document_topics(x,per_word_topics=True)[0])

In [ ]:
def KL_Divergence(x):
    a = x["lda_topicss1"]
    b = x["lda_topicss2"]
#     a_list=[]
#     b_list=[]
#     for i,j in a:
#         a_list.append(j)

#     for i,j in b:
#         b_list.append(j)
    
    a_list=np.array(a)
    b_list=np.array(b)
    return np.sum(np.where(a_list!=0,a_list*np.log(a_list/b_list),0))

data["kl_divergence"]= 0
data["kl_divergence"]= data.apply(KL_Divergence,axis=1)


In [ ]:
# features_to_use to train svm
 
features_to_use=[]

# for i in range(1,101):
#     features_to_use.append("docEmb1_"+str(i))
#     features_to_use.append("docEmb2_"+str(i))    
#     features_to_use.append("wordEmb1_"+str(i))    
#     features_to_use.append("wordEmb2_"+str(i))

features_to_use.append("doc_cosine")    
features_to_use.append("word_cosine")    
features_to_use.append("SC_score")  
features_to_use.append("Novelty_score")
features_to_use.append("kl_divergence")



In [ ]:
# spliting train and test data
Finaldata = shuffle(data,random_state = 2019)

X = Finaldata[features_to_use]
y = Finaldata["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,shuffle = False)


In [ ]:
Finaldata.label.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
print("Size of Train :",len(X_train))
print("Size of Test :",len(X_test))

In [ ]:
# training svm on training data
clf =  OneVsRestClassifier(svm.SVC(probability=True,verbose = True),n_jobs=-1)
clf.fit(X_train, y_train)



In [ ]:
# predicting probability on test data
y_pred = clf.predict_proba(X_test)
y_pred


In [ ]:
# predcition on test data
y_pred = clf.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt="d")

In [ ]:
# accuracy on test data
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
# logloss on test data
log_loss(y_test,y_pred)

In [ ]:
# saving the svm model
filename = './Final_Data/Science_SVM_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 

In [ ]:
# saving the data file
Finaldata.to_csv("./Final_Data/Science_SVM_Data.csv",index=False)

In [ ]:
# # loading svm model
# pickle_in = open("./data/Science_SVM_model.sav","rb")
# clf = pickle.load(pickle_in)
